In [5]:
#Import all libraries

import findspark

# initialize your spark directory
findspark.init('C:\opt\spark') 


# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from textblob import TextBlob
from collections import namedtuple
import time

In [2]:
# Creating the Spark Context
sc = SparkContext(master="local[2]", appName="WindowWordCount")
sc.setLogLevel("ERROR")

#creating the streaming context
ssc = StreamingContext(sc, 10)
ssc.checkpoint("checkpoint")

#creating the SQL context
sqlContext = SQLContext(sc)


lines = ssc.socketTextStream("127.0.0.1", 5555)

#Function to clean tweet
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", str(tweet)).split())


#Polarity analysis of a tweet
def analyze_sentiment_polarity(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    # return str(analysis.sentiment.polarity)
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

#subjectivity analysis of a tweet
def analyze_sentiment_subjectivity(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    return str(analysis.sentiment.subjectivity)


#Transforming using basic spark functions
sentiment = lines.map(lambda text: (text, analyze_sentiment_polarity(text), analyze_sentiment_subjectivity(text)))
words = lines.flatMap(lambda text: text.split(" ")).filter(lambda text: text.lower().startswith('#'))

fields = ("hashtags", "count")
Tagscount = namedtuple('Tagscount', fields)

sentimentfields = ("text", "polarity", "subjectivity")
Sentimentobject = namedtuple('Sentimentobject', sentimentfields)

sentiment.window(180, 60).map(lambda p: Sentimentobject(p[0], p[1], p[2])).foreachRDD(
    lambda rdd: rdd.toDF().registerTempTable("sentiment"))
words.countByValueAndWindow(180, 60).map(lambda p: Tagscount(p[0], p[1])).foreachRDD(
    lambda rdd: rdd.toDF().sort(desc('count')).limit(10).registerTempTable("hashtags"))

In [3]:
ssc.start()

In [6]:
print("Session Started.....")
print("Collecting tweets...waiting for 60 seconds..")

time.sleep(20)
print("Tweets Collected....")

#Connecting to a boto3 session - Update file in AWS Cloud S3 Bucket

pos_sentiment = sqlContext.sql('Select count(text) from sentiment where polarity = 1')
neu_sentiment = sqlContext.sql('Select count(text) from sentiment where polarity = 0')
neg_sentiment = sqlContext.sql('Select count(text) from sentiment where polarity = -1')

pos_df = pos_sentiment.toPandas()
neu_df = neu_sentiment.toPandas()
neg_df = neg_sentiment.toPandas()

Session Started.....
Tweets Collected....


AnalysisException: 'Table or view not found: sentiment; line 1 pos 24'